In [1]:
import numpy as np
import tensorflow as tf
from tensorflow import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.utils import np_utils
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Input
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import cv2
from tensorflow.keras import layers, regularizers

In [2]:
# Load Data Here
X = np.load('data_train.npy')
Y = np.load('t_train_cor_formatted.npy')

In [3]:
def train(data_load, train_load):
    '''
    Data Preprocessing
    '''
    # Load data
    X_early = data_load
    t_early = train_load
    
    # Reshape data
    D = 50
    X_early = np.array([cv2.resize(x.reshape(300,300), (D,D)).reshape(D*D) 
    for x in np.transpose (X_early) ])

    # Cut out the flattens (-2) and unknown (-1)
    X_train = np.delete(np.transpose(X_early), np.where(t_early < 0), axis=1)
    t_train = np.delete(t_early, np.where(t_early < 0))
    
    # Partition the training and test data
    x_train, x_test, y_train, y_test = train_test_split(X_train.T, t_train, test_size=0.2, random_state=42)
    
    # Reshape data for inputing into model
    x_train = np.reshape(x_train, (x_train.shape[0], D, D))
    x_test = np.reshape(x_test, (x_test.shape[0], D, D))
    
    
    '''
    Model Creation
    '''
    # Model & data parameters
    num_classes = 10
    variable_learning_rate = tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', factor = 0.2, patience = 2)
    tf.random.set_seed(2)
    model = Sequential([
        keras.Input(shape=(D, D, 1)),

        # Layer 1
        layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, activation = 'relu', input_shape = (D, D, 1), kernel_regularizer=regularizers.l2(0.0005)),
        # Layer 2
        layers.Conv2D(filters = 32, kernel_size = 5, strides = 1, use_bias=False),
        # Layer 3
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size = 2, strides = 2),
        layers.Dropout(0.25),
        # — — — — — — — — — — — — — — — — #
        # Layer 4
        layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005)),
        # Layer 5
        layers.Conv2D(filters = 64, kernel_size = 3, strides = 1, use_bias=False),
        # Layer 6
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size = 2, strides = 2),
        layers.Dropout(0.25),
        # — — — — — — — — — — — — — — — — #
        # Layer 7
        layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, activation = 'relu', kernel_regularizer=regularizers.l2(0.0005)),
        # Layer 8
        layers.Conv2D(filters = 128, kernel_size = 3, strides = 1, use_bias=False),
        # Layer 9
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        layers.MaxPooling2D(pool_size = 2, strides = 2),
        layers.Dropout(0.25),
        layers.Flatten(),
        # — — — — — — — — — — — — — — — — #
        # Layer 10
        layers.Dense(units = 256, use_bias=False),
        # Layer 11
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        # — — — — — — — — — — — — — — — — #
        # Layer 12
        layers.Dense(units = 128, use_bias=False),
        # Layer 13
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        # — — — — — — — — — — — — — — — — #
        # Layer 14
        layers.Dense(units = 84, use_bias=False),
        # Layer 15
        layers.BatchNormalization(),
        # — — — — — — — — — — — — — — — — #
        layers.Activation("relu"),
        layers.Dropout(0.25),
        # — — — — — — — — — — — — — — — — #
        # Output
        layers.Dense(units = 10, activation = 'softmax')
        ])

    model.summary()
    
    '''
    Model Compilation
    '''
    batch_size = 32
    epochs = 30

    adamopt = keras.optimizers.Adam(learning_rate=0.001)
    model.compile(loss="sparse_categorical_crossentropy", optimizer="adam", metrics=["accuracy"])
    
    
    '''
    Model Training
    '''
    model.fit(x_train, y_train, batch_size=batch_size, callbacks=[variable_learning_rate], epochs=epochs, validation_split=0.3)
    
    
    '''
    Model Evaluation
    '''
    print('Model Evaluation:')
    model.evaluate(x_train,y_train)
    model.evaluate(x_test,y_test)
    
    '''
    Save Model
    '''
    model.save("teaml_final_test_model.h5")

In [4]:
train(X,Y)

2022-12-07 21:22:06.694665: I tensorflow/core/platform/cpu_feature_guard.cc:151] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2022-12-07 21:22:07.214227: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1525] Created device /job:localhost/replica:0/task:0/device:GPU:0 with 79111 MB memory:  -> device: 0, name: NVIDIA A100-SXM4-80GB, pci bus id: 0000:87:00.0, compute capability: 8.0


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 46, 46, 32)        832       
                                                                 
 conv2d_1 (Conv2D)           (None, 42, 42, 32)        25600     
                                                                 
 batch_normalization (BatchN  (None, 42, 42, 32)       128       
 ormalization)                                                   
                                                                 
 activation (Activation)     (None, 42, 42, 32)        0         
                                                                 
 max_pooling2d (MaxPooling2D  (None, 21, 21, 32)       0         
 )                                                               
                                                                 
 dropout (Dropout)           (None, 21, 21, 32)        0

2022-12-07 21:22:09.468353: I tensorflow/stream_executor/cuda/cuda_dnn.cc:366] Loaded cuDNN version 8201
2022-12-07 21:22:10.311934: W tensorflow/stream_executor/gpu/asm_compiler.cc:80] Couldn't get ptxas version string: INTERNAL: Running ptxas --version returned 32512
2022-12-07 21:22:10.389191: W tensorflow/stream_executor/gpu/redzone_allocator.cc:314] INTERNAL: ptxas exited with non-zero error code 32512, output: 
Relying on driver to perform ptx compilation. 
Modify $PATH to customize ptxas location.
This message will be only logged once.
2022-12-07 21:22:11.297196: I tensorflow/stream_executor/cuda/cuda_blas.cc:1774] TensorFloat-32 will be used for the matrix multiplication. This will only be logged once.


156/156 [==============================] - 5s 7ms/step - loss: 2.0405 - accuracy: 0.3198 - val_loss: 1.5263 - val_accuracy: 0.5183 - lr: 0.0010
Epoch 2/30
156/156 [==============================] - 1s 4ms/step - loss: 1.1215 - accuracy: 0.6710 - val_loss: 1.3427 - val_accuracy: 0.5947 - lr: 0.0010
Epoch 3/30
156/156 [==============================] - 1s 4ms/step - loss: 0.8658 - accuracy: 0.7411 - val_loss: 1.3695 - val_accuracy: 0.6111 - lr: 0.0010
Epoch 4/30
156/156 [==============================] - 1s 4ms/step - loss: 0.7480 - accuracy: 0.7830 - val_loss: 1.1188 - val_accuracy: 0.6860 - lr: 0.0010
Epoch 5/30
156/156 [==============================] - 1s 4ms/step - loss: 0.6687 - accuracy: 0.8021 - val_loss: 1.5844 - val_accuracy: 0.5651 - lr: 0.0010
Epoch 6/30
156/156 [==============================] - 1s 4ms/step - loss: 0.6097 - accuracy: 0.8262 - val_loss: 0.6863 - val_accuracy: 0.7896 - lr: 0.0010
Epoch 7/30
156/156 [==============================] - 1s 4ms/step - loss: 0.5596 